In [0]:
# Install Unity Catalog AI integration packages with the Databricks extra
%pip install -q unitycatalog-ai[databricks]
%pip install -q unitycatalog-langchain[databricks]

# Install the Databricks LangChain integration package
%pip install -q databricks-langchain

dbutils.library.restartPython()

In [0]:
def get_columns(table_name:str, column_name:str) -> str:
    """
    A function that accepts a table name and a column name and returns all of the unique values in the given column for the given table

    Args:
        table_name (str): the fully qualified name of the table, in the format 'catalog'.'schema'.'table'
        column_name (str): the name of the column

    Returns:
        str: a string of the unique values in the given column for the given table
    """
    import os
    from databricks.sdk import WorkspaceClient
    
    query_text = f"""SELECT DISTINCT {column_name} FROM {table_name}"""
    
    try:
      w = WorkspaceClient(host=os.environ["DATABRICKS_HOST"], token=os.environ["DATABRICKS_TOKEN"])
      results = w.statement_execution.execute_statement(query_text, warehouse_id=os.environ["DATABRICKS_WAREHOUSE_ID"])

      results_list = [f"`{t[0]}`" for t in results.result.data_array]
      results_output = ',\n'.join(results_list)

      return results_output
    except Exception as e:
      raise Exception(f"Error executing query: {e}")
      pass

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

CATALOG = "doan"
SCHEMA = "bakehouse_text2sql"

client = DatabricksFunctionClient()

function_info = client.create_python_function(
  func=get_columns,
  catalog=CATALOG,
  schema=SCHEMA,
  replace=True,
  dependencies=["databricks-sdk"]
)

In [0]:
result = client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.get_columns",
    parameters={"table_name": "doan.bakehouse_text2sql.sales_suppliers", "column_name": "ingredient"}
)

result.value